In [1]:
%pylab inline
import pandas
import sqlite3
import pymc3
import seaborn as sns
from sklearn import *
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/riri/.virtualenvs/mxnet-3/lib/python3.4/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [3]:
con = sqlite3.connect('/tmp/data.sqlite3')

In [ ]:
pandas.read_sql('SELECT rand, count(*) FROM data group by 1', con=con)

In [ ]:
pandas.read_sql('SELECT week_num, count(*) FROM data group by 1', con=con)

In [ ]:
sample = pandas.read_sql('SELECT * FROM data where sales_depo < 1200 and adjusted_demand is not null', con=con)

In [ ]:
print(sample.shape)
sample.dtypes

In [ ]:
def aggregate_field(sample, field_name):
    return pandas.merge(
        sample, 
        sample.groupby(field_name).adjusted_demand.median().rename(field_name + '_median').to_frame(),
        left_on=field_name, right_index=True).drop(field_name, axis=1)

x = sample
x = aggregate_field(x, 'week_num')
x = aggregate_field(x, 'sales_depo')
x = aggregate_field(x, 'sales_channel')
x = aggregate_field(x, 'route_id')
x = aggregate_field(x, 'client_id')
x = aggregate_field(x, 'product_id')
x = x.drop(['id', 'week_sales', 'week_returns', 'next_week_returns', 'next_week_returns_peso', 'rand'], axis=1)

In [ ]:
manif = manifold.TSNE()
samp = x.sample(1000)
loc = manif.fit_transform(samp.drop('adjusted_demand', axis=1))
figure(figsize=(10, 10))
scatter(*loc.T, cmap='viridis', c=np.log(samp.adjusted_demand + 1))

In [6]:
some_things = pandas.read_sql('SELECT product_id FROM data where rand=10 limit 100', con=con).product_id.unique()

In [7]:
some_things.shape

(52,)

In [12]:
prod_ints = ','.join([str(s) for s in some_things])

In [ ]:
prod_info = pandas.read_sql('''
    SELECT d.*
      FROM data d
     WHERE d.product_id IN ({})
'''.format(prod_ints), con=con)

In [ ]:
means = prod_info.groupby(['week_num', 'client_id']).adjusted_demand.mean().unstack().dropna()
means.div(means.iloc[0]).plot()

In [ ]:
means